In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

In [2]:
#checking if Cuda is availble or not. CUDA enables developers to speed up compute-intensive 
#applications by harnessing the power of GPUs for the parallelizable part of the computation. 

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print (f'device:{device}')

# Hyper-paramesters

#defining the number of epochs and batch size. Batch size is dividing the datasets into parts whatever you define 
num_epochs = 8
batch_size = 2
learning_rate = 0.001

device:cuda


In [3]:
# dataset has PILImage images of rang [0,1]
# we transform them to Tensors of normalized range [-1,1]

transform = transforms.Compose(
    [transforms.ToTensor(),  #transforming it to Tensors 
     transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]) #normalizing it
 
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [5]:
VGG_types = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
}

In [6]:
class VGG_net(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(VGG_net, self).__init__()
        self.in_channels = in_channels
        self.conv_layers = self.create_conv_layer(VGG_types['VGG19'])
                
        self.fcs = nn.Sequential(
            nn.Linear(512, 4096),  # 32/32 = 1 
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(),
            nn.Dropout(p=0.5),
            nn.Linear(4096, num_classes)
        )
        
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fcs(x)
        return x
    
    def create_conv_layer(self, architecture):
        layers = []
        in_channels = self.in_channels
        
        for x in architecture:
            if type(x) == int:
                out_channels = x
                layers += [nn.Conv2d(in_channels = in_channels, out_channels = out_channels,
                                    kernel_size=(3,3), stride=(1,1), padding=(1,1)),
                           nn.BatchNorm2d(x), nn.ReLU()]
                in_channels = x
            elif x == 'M':
                layers += [nn.MaxPool2d(kernel_size=(2,2), stride = (2,2))]
        
        return nn.Sequential(*layers)

In [ ]:
model = VGG_net(in_channels=3, num_classes = 10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
 
n_total_steps = len(train_loader)
t1 = dt.datetime.now()
print (f'Training started at {t1}')
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
 
        outputs = model(images)
        
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2000 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], loss: {loss.item():.4f}')
 
t2 = dt.datetime.now()
print(f'Finished Training at {t2}')
print(f'Training time :  {t2-t1}')
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range (batch_size):
            label = labels[i]
            pred = predicted[i]
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    acc = 100.0 * n_correct / n_samples
    print (f'Accuracy of the network: {acc} %')
    
    for i in range(10):
        acc = 100.0 * n_class_correct[i] / n_class_samples[i]
        print (f'Accuracy of {classes[i]}: {acc} %')

Training started at 2021-05-26 18:44:47.727900
Epoch [1/8], Step [2000/25000], loss: 2.0536
Epoch [1/8], Step [4000/25000], loss: 2.2718
Epoch [1/8], Step [6000/25000], loss: 1.4799
Epoch [1/8], Step [8000/25000], loss: 1.8100
Epoch [1/8], Step [10000/25000], loss: 1.7516
Epoch [1/8], Step [12000/25000], loss: 1.8268
Epoch [1/8], Step [14000/25000], loss: 1.4489
Epoch [1/8], Step [16000/25000], loss: 0.9809
Epoch [1/8], Step [18000/25000], loss: 1.5223
Epoch [1/8], Step [20000/25000], loss: 1.8953
Epoch [1/8], Step [22000/25000], loss: 2.4797
Epoch [1/8], Step [24000/25000], loss: 0.6786
Epoch [2/8], Step [2000/25000], loss: 0.8704
Epoch [2/8], Step [4000/25000], loss: 0.3169
Epoch [2/8], Step [6000/25000], loss: 2.6658
Epoch [2/8], Step [8000/25000], loss: 1.6311
Epoch [2/8], Step [10000/25000], loss: 1.0480
Epoch [2/8], Step [12000/25000], loss: 1.7122
Epoch [2/8], Step [14000/25000], loss: 2.0180
Epoch [2/8], Step [16000/25000], loss: 0.6934
Epoch [2/8], Step [18000/25000], loss: 1.